In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

np.random.seed(1)
random.seed(1)

src_dir = '/content/drive/MyDrive/captcha_images'
img_paths = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg'))]

#view an example CAPTCHA
img = cv2.imread(os.path.join(src_dir, img_paths[0]))
plt.imshow(img)
plt.title(f'Shape: {img.shape}')
plt.show()

#get list of ground-truth labels from image filenames
labels = [img_path.split('.')[0] for img_path in img_paths]

#get count of each unique char
char_counts = {}
for label in labels:
    for char in label:
        if char not in char_counts:
            char_counts[char] = 1
        else:
            char_counts[char] += 1

#sort by character
char_counts = sorted(char_counts.items())
#convert items to two stand-alone lists
chars = [char_count[0] for char_count in char_counts]
counts = [char_count[1] for char_count in char_counts]
#print(chars)
chars.reverse()
counts.reverse()
plt.figure(figsize=(10, 10))

bars = plt.barh(y=chars, width=counts, height=0.6)
for bar in bars:
    print(bar)
    w = bar.get_width()
    y = bar.get_y()
    h = bar.get_height()
    plt.text(w+40, y+h/2, w, ha='center', va='center')
    #print()

plt.xlim(0, 1500)
plt.grid(linestyle='-', axis='x')
plt.title(f'Character counts in CAPTCHA labels ({len(chars)} unique characters)')
plt.show()
print(chars)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##character segmentation

In [ ]:
fig = plt.figure(figsize=(14, 10))

plt_rows = 4
plt_cols = 2
plt_iter = 1

plt.subplots_adjust(hspace=0.5)

for i in range(plt_rows*plt_cols):
    plt.subplot(plt_rows, plt_cols, plt_iter)

    img_index = np.random.randint(0, len(img_paths))
    img = cv2.imread(os.path.join(src_dir, img_paths[img_index]))                 #load random image
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                              #convert to grayscale
    ret, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)            #take binary threshold
    bit_not = cv2.bitwise_not(thresh)                                             #invert image
    contours, hierarchy = cv2.findContours(bit_not, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)   #find contours

    #draw on orignal image
    cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
    #get bounding rect of each contour
    rects = [cv2.boundingRect(c) for c in contours]
    #sort rects by their width
    rects.sort(key=lambda x: x[2])

    #deal with touching letters where one wide bounding box envlopes two letters. split these in half
    while len(rects) < 4:
        #pop widest rect
        wide_rect = rects.pop()
        x, y, w, h = wide_rect
        #split in two
        first_half = (x, y, w//2, h)
        second_half = (x+w//2, y, w//2, h)
        rects.append(first_half)
        rects.append(second_half)
        #re-sort rects by their width
        rects.sort(key=lambda x: x[2])

    for rect in rects:
        x, y, w, h = rect
        #buffer rect by 1 pixel
        cv2.rectangle(img, (x-1, y-1), (x+w+1, y+h+1), (255, 0, 0), 1)

    plt.imshow(img)
    plt_iter += 1


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import time

np.random.seed(1)
random.seed(1)

src_dir = '/content/drive/MyDrive/captcha_images'
img_paths = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg'))]

# Loop through the images and extract and display individual characters
for img_path in img_paths:
    img = cv2.imread(os.path.join(src_dir, img_path))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    bit_not = cv2.bitwise_not(thresh)
    contours, hierarchy = cv2.findContours(bit_not, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Get bounding rectangles for the characters
    rects = [cv2.boundingRect(c) for c in contours]
    rects.sort(key=lambda x: x[0])  # Sort by the x-coordinate to ensure correct character order

    # Loop through each character and display it
    for i, rect in enumerate(rects):
        x, y, w, h = rect
        char_image = img[y:y+h, x:x+w]
        plt.subplot(1, len(rects), i + 1)
        plt.imshow(char_image, cmap='gray')
        plt.title(f'Character {i+1}')
        plt.axis('off')

    plt.show()

    start_time = time.time()
    end_time = time.time()  # Record the end time for performance measurement
    elapsed_time = end_time - start_time
    print(f"Processing time for image {img_path}: {elapsed_time:.2f} seconds")

# Calculate the total time for processing all images
end_time = time.time()
total_elapsed_time = end_time - start_time
print(f"Total processing time for all images: {total_elapsed_time:.2f} seconds")